In [2]:
# Mount drive:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Import

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
tqdm.pandas()
from sklearn.preprocessing import OneHotEncoder


C:\Users\adida\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Functions

In [2]:
def write_data(df, full_url):
    df.to_csv(full_url, index=False)
    print('Saved csv in {u}'.format(u=str(full_url)))


def get_value_counts(df, col_name):

  value_counts_df = df[col_name].value_counts().reset_index()
  value_counts_df = value_counts_df.rename(columns={'index':col_name,col_name: 'count'})
  value_counts_df = value_counts_df.sort_values(by='count',ascending=False)
  return value_counts_df


def plot_histogram(df, col_name, title, hue_col_name=None):

  if hue_col_name:
    ax = sns.displot(df, x=col_name, hue=hue_col_name)
  else:
    ax = sns.histplot(df[col_name])
  
  plt.suptitle(title)
  plt.show()


def flatten(nested_list):
    for i in nested_list:
        if isinstance(i, (list, tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i


def flatten_list(nested_list):
    return list(flatten(nested_list))


def read_data(full_url):
    df = pd.read_csv(full_url, low_memory=False)
    print('Upload csv from {u}'.format(u=str(full_url)))

    return df


def concat_dfs_by_row(list_of_dfs):
    new_list_Of_dfs = []

    if len(list_of_dfs) < 2:
        print_error('Error: less than two DFs.')
        return None

    col_shape = list_of_dfs[0].shape[1]
    for df in tqdm(list_of_dfs):
        assert df.shape[1] == col_shape
        df.reset_index(drop=True, inplace=True)
        new_list_Of_dfs.append(df)

    all_df = pd.concat(new_list_Of_dfs, axis=0, ignore_index=True)

    return all_df


def filter_transactions(membership_expire_date, churn_date):

  if membership_expire_date < churn_date:
    return 'Yes'
  else:
    return 'No'

def split_date(df,col_name):
    df = df.copy()
    df[col_name+"_"+"Year"] = df[col_name].apply(pd.to_datetime).dt.year
    df[col_name+"_"+"Month"] = df[col_name].apply(pd.to_datetime).dt.month
    df[col_name+"_"+"Day"] = df[col_name].apply(pd.to_datetime).dt.day

    return df

def one_hot_encoder(df,col_name):
    dummy = pd.get_dummies(df[col_name])
    df = pd.concat((df,dummy), axis=1)
    df = df.drop([col_name], axis=1)
    print(df)

    return df  

# Config

In [3]:
project_url = "drive/MyDrive/DataScience/project/"

# Data Preprocessing

## Load Data

In [ ]:
train = pd.read_csv(project_url + "train.csv")
train_v2 = pd.read_csv(project_url + "train_v2.csv")

In [ ]:
train['churn_month'] = 'FEB'
train_v2['churn_month'] = 'MAR'
train['churn_date'] = pd.to_datetime(datetime.strptime(str(int(20170301)), "%Y%m%d"))
train_v2['churn_date'] = pd.to_datetime(datetime.strptime(str(int(20170401)), "%Y%m%d"))

In [ ]:
full_train = pd.concat([train, train_v2], ignore_index=True)

In [ ]:
sample_submission_zero = pd.read_csv(project_url + "sample_submission_zero.csv")
sample_submission_zero_v2 = pd.read_csv(project_url + "sample_submission_v2.csv")

In [ ]:
sample_submission_zero['churn_month'] = 'MAR'
sample_submission_zero_v2['churn_month'] = 'APR'

In [ ]:
sample_submission_zero['churn_date'] = pd.to_datetime(datetime.strptime(str(int(20170401)), "%Y%m%d"))
sample_submission_zero_v2['churn_date'] = pd.to_datetime(datetime.strptime(str(int(20170501)), "%Y%m%d"))

In [ ]:
test = pd.concat([sample_submission_zero, sample_submission_zero_v2], ignore_index=True)

In [ ]:
transactions = pd.read_csv(project_url + "transactions.csv")
transactions_v2 = pd.read_csv(project_url + "transactions_v2.csv")

In [ ]:
full_transactions = pd.concat([transactions, transactions_v2], ignore_index=True)

In [ ]:
members = pd.read_csv(project_url + "members_v3.csv")

In [9]:
users_logs = pd.read_csv(project_url + "users_logs_30M.csv")
user_logs_v2 = pd.read_csv(project_url + "user_logs_v2.csv")

In [10]:
full_user_logs = pd.concat([users_logs, user_logs_v2], ignore_index=True)

# Data Preprocessing

## Train

In [ ]:
full_train = full_train.drop_duplicates()
full_train['msno'] = full_train['msno'].astype(str)

In [ ]:
# Check for duplicates
full_train.dropna().shape[0] == full_train.shape[0]

True

In [ ]:
full_train.dtypes

msno                   object
is_churn                int64
churn_month            object
churn_date     datetime64[ns]
dtype: object

In [ ]:
# Count 0 and 1 in 'is_churn' column
Counter(full_train['is_churn'])

Counter({0: 1813090, 1: 150801})

In [ ]:
(139749/(139749+988431))*100

12.387119076743073

In [ ]:
# write_data(df=full_train, full_url=project_url+'Data/full_train.csv')

Saved csv in drive/MyDrive/project/Data/full_train.csv


In [5]:
full_train = read_data(full_url=project_url+'Data/full_train.csv')

Upload csv from drive/MyDrive/DataScience/project/Data/full_train.csv


## Test

In [ ]:
# 'is_churn' column in test containing only 0. Therefore, we will drop the column.
test = test.drop('is_churn',axis=1)

In [ ]:
# Convert user_id ('msno') from Object to str.
test['msno'] = test['msno'].astype(str)

In [ ]:
# Cיheck for duplicates
test.dropna().shape[0] == test.shape[0]

True

In [ ]:
# write_data(df=test, full_url=project_url+'Data/test.csv')

In [6]:
test = read_data(full_url=project_url+'Data/test.csv')

Upload csv from drive/MyDrive/DataScience/project/Data/test.csv


## Transactions

In [ ]:
# convert user_id ('msno') to str
full_transactions['msno'] = full_transactions['msno'].astype(str)

In [ ]:
# Convert dates format.
full_transactions['transaction_date'] = full_transactions['transaction_date'].apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )
full_transactions['membership_expire_date'] = full_transactions['membership_expire_date'].apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )

In [ ]:
# Check num of rows
full_transactions.shape[0]

22978755

In [ ]:
# Check for NA(s)
full_transactions.isna().sum()

msno                      0
payment_method_id         0
payment_plan_days         0
plan_list_price           0
actual_amount_paid        0
is_auto_renew             0
transaction_date          0
membership_expire_date    0
is_cancel                 0
dtype: int64

In [ ]:
users = []
train_users = list(full_train['msno'])
test_users = list(test['msno'])
users.append(train_users)
users.append(test_users)
users = flatten_list(nested_list=users)

In [ ]:
len(users)

3842322

In [ ]:
# Subset to train and test users only:
full_transactions = full_transactions[full_transactions['msno'].isin(users)]

In [ ]:
full_transactions.shape[0]

17861218

In [ ]:
# write_data(df=full_transactions, full_url=project_url+'Data/full_transactions.csv')

Saved csv in drive/MyDrive/project/Data/full_transactions.csv


In [ ]:
full_transactions = read_data(full_url=project_url+'Data/full_transactions.csv')

Upload csv from drive/MyDrive/DataScience/project/Data/full_transactions.csv


In [ ]:
# Filter transactions per churn date:
full_train['trainOrTest'] = 'Train'
test['trainOrTest'] = 'Test'
target_df = pd.concat([full_train, test], ignore_index=True)

In [ ]:
full_transactions = pd.merge(full_transactions, target_df, on='msno', how='left')
full_transactions['membership_expire_date'] = pd.to_datetime(full_transactions['membership_expire_date'])

In [ ]:
pos_full_transactions = full_transactions[full_transactions['is_churn']==1]
not_pos_full_transactions = full_transactions[full_transactions['is_churn']!=1]
not_pos_full_transactions['keep'] = 'Yes'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
del full_transactions

In [ ]:
pos_full_transactions['churn_date'] = pd.to_datetime(pos_full_transactions['churn_date'])

In [ ]:
pos_full_transactions['keep'] = pos_full_transactions.progress_apply(lambda x: filter_transactions(membership_expire_date=x['membership_expire_date'],
                                                                                                    churn_date=x['churn_date']), axis=1)

100%|██████████| 1817877/1817877 [00:51<00:00, 35548.11it/s]


In [ ]:
pos_full_transactions = pos_full_transactions[pos_full_transactions['keep']=='Yes'].reset_index(drop=True)

In [ ]:
pos_full_transactions = pos_full_transactions.drop(['is_churn','churn_month','trainOrTest','keep'],axis=1)

In [ ]:
write_data(df=pos_full_transactions, full_url=project_url+'Data/pos_full_transactions.csv')

Saved csv in drive/MyDrive/project/Data/pos_full_transactions.csv


In [ ]:
not_pos_full_transactions.head(4)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,is_churn,churn_month,churn_date,trainOrTest,keep
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,2015-09-30,2015-11-01,0,0.0,FEB,2017-03-01,Train,Yes
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0,0.0,FEB,2017-03-01,Train,Yes
2,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0,0.0,MAR,2017-04-01,Train,Yes
3,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0,NaN,MAR,2017-04-01,Test,Yes


In [ ]:
not_pos_full_transactions = not_pos_full_transactions.drop(['is_churn','churn_month','trainOrTest','keep'],axis=1)

In [ ]:
write_data(df=not_pos_full_transactions, full_url=project_url+'Data/not_pos_full_transactions.csv')

Saved csv in drive/MyDrive/project/Data/not_pos_full_transactions.csv


In [ ]:
pos_full_transactions = read_data(full_url=project_url+'Data/pos_full_transactions.csv')

Upload csv from drive/MyDrive/project/Data/pos_full_transactions.csv


In [ ]:
not_pos_full_transactions = read_data(full_url=project_url+'Data/not_pos_full_transactions.csv')

In [ ]:
full_transactions = concat_dfs_by_row(list_of_dfs=[pos_full_transactions, not_pos_full_transactions])

100%|██████████| 2/2 [00:00<00:00, 2619.80it/s]


In [ ]:
full_transactions.shape[0]

63921677

In [ ]:
write_data(df=full_transactions, full_url=project_url+'Data/full_transactions.csv')

Saved csv in drive/MyDrive/project/Data/full_transactions.csv


In [ ]:
full_transactions = read_data(full_url=project_url+'Data/full_transactions.csv')

Upload csv from drive/MyDrive/DataScience/project/Data/full_transactions.csv


In [ ]:
full_transactions

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,2015-09-30,2015-11-01,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0
2,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,2015-09-30,2015-11-21,0
3,KN7I82kjY0Tn76Ny95ncqBUVbO7b8AXrOPqZutKpxIM=,21,30,149,149,1,2015-09-30,2015-11-07,0
4,m5ptKif9BjdUghHXXomSezy5ohJiHm85PE13f/3kQaw=,39,30,149,149,1,2015-09-30,2015-11-28,0
...,...,...,...,...,...,...,...,...,...
17861213,zviO0RLvsXAzgz894wMm5b9Nxp8yxFAuqHY1Nq0lq3M=,36,30,180,180,1,2017-03-16,2017-04-15,0
17861214,zwF50wwaJI2TBKWhB42HRBJ6EQK0jgSo1Xmwb9Jq3SU=,32,180,536,536,0,2017-02-15,2017-08-17,0
17861215,zx/h5MzQQmsSat04wSfGpHp6N8aWLLwM1+7OV7ujmPY=,41,30,149,149,1,2017-03-06,2017-04-06,0
17861216,zxvgjIKjy18Fm+cIWUfYKr68z09+ILBxuMW0DnbeUZ8=,41,30,99,99,1,2017-03-08,2017-04-08,0


In [ ]:
example = full_transactions[full_transactions['msno'] == 'Xik55GiZKTkU1fMHNuROOSIRjf67/9HQ9xYn7Ltldmk=']

In [ ]:
example['is_auto_renew'].value_counts().values[0]

19

In [ ]:
example2['count'] = example.groupby('msno', as_index = False).agg({'is_auto_renew_count': lambda x:x('is_auto_renew').value_counts().values[0]})

SpecificationError: ignored

In [ ]:
full_transactions[full_transactions['msno'] == 'Xik55GiZKTkU1fMHNuROOSIRjf67/9HQ9xYn7Ltldmk='].

In [ ]:
transactions_v3[transactions_v3['msno'] == 'Xik55GiZKTkU1fMHNuROOSIRjf67/9HQ9xYn7Ltldmk=']

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
638184,Xik55GiZKTkU1fMHNuROOSIRjf67/9HQ9xYn7Ltldmk=,30,31,149,149,1,2015-01-07,2017-04-22,0


In [ ]:
transactions_v3 = full_transactions.groupby('msno', as_index = False).agg({'payment_method_id': lambda x:x.value_counts().index[0], 'payment_plan_days': 'max', 'plan_list_price': 'max',
                                       'actual_amount_paid': 'max', 'is_auto_renew': lambda x:x.value_counts().index[0], 'transaction_date': 'min', 'membership_expire_date': 'max',
                                       'is_cancel': lambda x:x.value_counts().index[0]})

In [ ]:
transactions_v3[]

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2016-11-16,2017-04-15,0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,31,149,149,1,2015-01-31,2017-05-19,0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,1,2015-01-26,2017-04-26,0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,1,2016-03-15,2017-04-15,0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,31,149,149,1,2015-01-31,2017-05-23,0
...,...,...,...,...,...,...,...,...,...
1143467,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30,180,180,1,2015-04-22,2017-04-10,0
1143468,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,31,149,149,1,2015-02-13,2017-05-24,0
1143469,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30,149,149,1,2015-01-04,2017-02-05,0
1143470,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,40,30,149,149,1,2017-02-05,2017-04-04,0


In [ ]:
# transactions_v3['is_auto_renew_popolur_count'] = full_transactions.groupby('msno', as_index = False).agg({'is_auto_renew': lambda x:x.value_counts().values[0]})

ValueError: ignored

In [ ]:
transactions_v3.drop_duplicates(subset='msno').shape[0] == transactions_v3.shape[0]

True

In [ ]:
transactions_v3.rename(columns = {"payment_method_id":"payment_method_id_most_common","payment_plan_days":"max_payment_plan_days","plan_list_price":"max_plan_list_price",
                     "actual_amount_paid":"max_actual_amount_paid",'is_auto_renew':'is_auto_renew_most_common','transaction_date':'min_transaction_date','membership_expire_date':'max_membership_expire_date',
                     "is_cancel":"is_cancel_most_common"}) 

,msno,payment_method_id_most_common,max_payment_plan_days,max_plan_list_price,max_actual_amount_paid,is_auto_renew_most_common,min_transaction_date,max_membership_expire_date,is_cancel_most_common
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2016-11-16,2017-04-15,0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,31,149,149,1,2015-01-31,2017-05-19,0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,1,2015-01-26,2017-04-26,0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,41,30,149,149,1,2016-03-15,2017-04-15,0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,39,31,149,149,1,2015-01-31,2017-05-23,0
...,...,...,...,...,...,...,...,...,...
1143467,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,41,30,180,180,1,2015-04-22,2017-04-10,0
1143468,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,39,31,149,149,1,2015-02-13,2017-05-24,0
1143469,zzz4xr5LLzoxUqD6LfKiieCt8Or4dUWFQ0RA5gmkx8I=,41,30,149,149,1,2015-01-04,2017-02-05,0
1143470,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,40,30,149,149,1,2017-02-05,2017-04-04,0


In [ ]:
#write_data(df=full_transactions, full_url=project_url+'Data/transactions_v3.csv')

Saved csv in drive/MyDrive/DataScience/project/Data/transactions_v3.csv


In [7]:
transactions_v3 = read_data(full_url=project_url+'Data/transactions_v3.csv')

Upload csv from drive/MyDrive/DataScience/project/Data/transactions_v3.csv


## Members

In [ ]:
members.shape[0]

6769473

In [ ]:
members['msno'] = members['msno'].astype(str)

In [ ]:
members.drop_duplicates(subset='msno').shape[0] == members.shape[0]

True

In [ ]:
members.isna().sum()

msno                            0
city                            0
bd                              0
gender                    4429505
registered_via                  0
registration_init_time          0
dtype: int64

In [ ]:
# Subset to train and test users only:
members = members[members['msno'].isin(users)]

In [ ]:
members.shape[0]

1018235

In [ ]:
members['registration_init_time'] = members['registration_init_time'].apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )

In [ ]:
members

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,2011-09-11
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,2011-09-14
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,2011-09-15
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,2011-09-15
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,2011-09-15
...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,0,NaN,7,2015-10-20
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,1,0,NaN,7,2015-10-20
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,15,26,female,4,2015-10-20
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,1,0,NaN,4,2015-10-20


In [ ]:
members

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,NaN,NaN,11,2011-09-11
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,NaN,NaN,7,2011-09-14
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,NaN,NaN,11,2011-09-15
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,NaN,NaN,11,2011-09-15
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32.0,female,9,2011-09-15
...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,NaN,NaN,7,2015-10-20
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,1,NaN,NaN,7,2015-10-20
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,15,26.0,female,4,2015-10-20
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,1,NaN,NaN,4,2015-10-20


In [ ]:
dummy = pd.get_dummies(members['gender'])

In [ ]:
dummy

,female,male
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
6769468,0,0
6769469,0,0
6769470,1,0
6769471,0,0


In [ ]:
members2 = pd.concat((members,dummy), axis=1)

In [ ]:
members2

,msno,city,bd,gender,registered_via,registration_init_time,female,male
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,2011-09-11,0,0
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,2011-09-14,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,2011-09-15,0,0
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,2011-09-15,0,0
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,2011-09-15,1,0
...,...,...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,0,NaN,7,2015-10-20,0,0
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,1,0,NaN,7,2015-10-20,0,0
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,15,26,female,4,2015-10-20,1,0
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,1,0,NaN,4,2015-10-20,0,0


In [69]:
members2 = members2.drop(['registration_init_time','gender'], axis=1)

In [ ]:
members2 = members2.rename(columns={"female":"is_female","male":"is_male"})

In [ ]:
members2

,msno,city,bd,gender,registered_via,registration_init_time,is_female,is_male
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,NaN,NaN,11,2011-09-11,0,0
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,NaN,NaN,7,2011-09-14,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,NaN,NaN,11,2011-09-15,0,0
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,NaN,NaN,11,2011-09-15,0,0
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32.0,female,9,2011-09-15,1,0
...,...,...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,NaN,NaN,7,2015-10-20,0,0
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,1,NaN,NaN,7,2015-10-20,0,0
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,15,26.0,female,4,2015-10-20,1,0
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,1,NaN,NaN,4,2015-10-20,0,0


In [13]:
# Clean bd (age):
lower_value = 10
higher_value = 120

In [ ]:
# Defined an acceptable range of ages for users in the workplace and replaced numbers outside of this range with null values.
members2.bd.loc[(members2.bd < lower_value) | (members2.bd > higher_value)] = np.nan

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
members2.isnull().mean()

msno                      0.000000
city                      0.000000
bd                        0.670917
gender                    0.654335
registered_via            0.000000
registration_init_time    0.000000
is_female                 0.000000
is_male                   0.000000
dtype: float64

In [ ]:
members['bd'].dtype

dtype('float64')

In [27]:
write_data(df=members2, full_url=project_url+'Data/members_v2.csv')

Saved csv in drive/MyDrive/DataScience/project/Data/members_v2.csv


In [8]:
members2 = read_data(full_url=project_url+'Data/members_v2.csv')

Upload csv from drive/MyDrive/DataScience/project/Data/members_v2.csv


In [63]:
members2 = split_date(df=members2,col_name='registration_init_time')

In [15]:
members2

,msno,city,bd,registered_via,is_female,is_male,registration_init_time_Year,registration_init_time_Month,registration_init_time_Day,AgeGroup
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,NaN,11,0,0,2011,9,11,NaN
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,NaN,7,0,0,2011,9,14,NaN
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,NaN,11,0,0,2011,9,15,NaN
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,NaN,11,0,0,2011,9,15,NaN
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32.0,9,1,0,2011,9,15,Age_25-34
...,...,...,...,...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,NaN,7,0,0,2015,10,20,NaN
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,1,NaN,7,0,0,2015,10,20,NaN
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,15,26.0,4,1,0,2015,10,20,Age_25-34
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,1,NaN,4,0,0,2015,10,20,NaN


In [14]:
# create age-groups
bins= [lower_value,20,25,35,45,55,65,higher_value]
labels = ['Age_10-19,','Age_20-24','Age_25-34','Age_35-44','Age_45-54','Age_55-65','Age_65+']
members2['AgeGroup'] = pd.cut(members2['bd'], bins=bins, labels=labels, right=False)

In [25]:
members2 = one_hot_encoder(df=members2,col_name='AgeGroup')

KeyError: ignored

In [26]:
members2

,msno,city,bd,registered_via,is_female,is_male,registration_init_time_Year,registration_init_time_Month,registration_init_time_Day,"Age_10-19,",Age_20-24,Age_25-34,Age_35-44,Age_45-54,Age_55-65,Age_65+
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,NaN,11,0,0,2011,9,11,0,0,0,0,0,0,0
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,NaN,7,0,0,2011,9,14,0,0,0,0,0,0,0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,NaN,11,0,0,2011,9,15,0,0,0,0,0,0,0
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,NaN,11,0,0,2011,9,15,0,0,0,0,0,0,0
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32.0,9,1,0,2011,9,15,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,NaN,7,0,0,2015,10,20,0,0,0,0,0,0,0
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,1,NaN,7,0,0,2015,10,20,0,0,0,0,0,0,0
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,15,26.0,4,1,0,2015,10,20,0,0,1,0,0,0,0
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,1,NaN,4,0,0,2015,10,20,0,0,0,0,0,0,0


## Users logs

In [11]:
# Check for duplicates
full_user_logs.dropna().shape[0] == full_user_logs.shape[0]

True

In [13]:
full_user_logs['date'] = full_user_logs['date'].apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )

In [14]:
full_user_logs

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-05-13,0,0,0,0,1,1,280.335
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-07-09,9,1,0,0,7,11,1658.948
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-01-05,3,3,0,0,68,36,17364.956
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-03-06,1,0,1,1,97,27,24667.317
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-05-01,3,0,0,0,38,38,9649.029
...,...,...,...,...,...,...,...,...,...
48396357,FGpiy2mB+vXLKziYRcY/xJcJEFJfRDfUqlU+p760f7E=,2017-03-14,0,0,0,0,1,1,248.058
48396358,iZRjKNMrw5ffEbfXODLhV/0tJLPbOH3am1WYDgqBf8Q=,2017-03-06,0,0,0,0,1,1,311.000
48396359,yztw4Y0EggG0w2wPkbMZx7ke7saSx7dLSfMheHZG/DQ=,2017-03-31,0,0,0,0,17,1,3973.189
48396360,swCHwkNx30/aENjq30qqaLlm7bUUytbMXdz1bH7g0Jk=,2017-03-07,0,0,0,1,0,1,179.278


In [34]:
full_user_logs_gb1= full_user_logs.groupby('msno').agg({'date':'count'}).reset_index().rename(columns={'date':'dates_count'})

In [35]:
full_user_logs_gb1

,msno,dates_count
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,4
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,74
2,+++dz9ZCWE2HB/47pJU82NJXQzQuZDx1Wm50YSk/kKk=,2
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,39
4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,64
...,...,...
2006679,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,4
2006680,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,11
2006681,zzztPAN9xjMytpZ0RN2gU9mScDULJnHQZK8eZb4uELU=,4
2006682,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,4


In [24]:
rslt_df_gb = rslt_df.groupby('msno', as_index = False).agg({'date': 'max', 'num_25': 'sum', 'num_50': 'sum', 'num_75': 'sum',
                                 'num_985': 'sum', 'num_100': 'sum', 'num_unq': 'max', 'total_secs': 'sum'})


In [39]:
user_logs_gb

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,2016-09-13,16,7,3,5,98,61,28081.624
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2017-03-31,139,29,26,20,5656,227,1388033.273
2,+++dz9ZCWE2HB/47pJU82NJXQzQuZDx1Wm50YSk/kKk=,2016-03-02,3,0,0,1,105,54,27344.668
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-03-31,233,113,92,194,776,59,253460.349
4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2017-03-31,85,34,40,41,1197,85,305064.873
...,...,...,...,...,...,...,...,...,...
2006679,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,2016-05-26,16,3,1,2,2,7,1532.633
2006680,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,2017-03-28,10,1,1,1,232,44,59525.679
2006681,zzztPAN9xjMytpZ0RN2gU9mScDULJnHQZK8eZb4uELU=,2015-02-09,9,7,1,2,0,5,1251.328
2006682,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,2015-04-22,2,0,0,4,16,5,5165.930


In [37]:
# returns the sum # of songs played less than X% of the song length
# returns the max value of date and number of unique songs
user_logs_gb = full_user_logs.groupby('msno', as_index = False).agg({'date': 'max', 'num_25': 'sum', 'num_50': 'sum', 'num_75': 'sum',
                                 'num_985': 'sum', 'num_100': 'sum', 'num_unq': 'max', 'total_secs': 'sum'})


In [40]:
user_logs_gb = pd.merge(user_logs_gb, full_user_logs_gb1, how='left', on='msno')

In [55]:
user_logs_gb

,msno,num_unq,total_secs,dates_count,percent_25,percent_50,percent_75,percent_985,percent_100,date_Year,date_Month,date_Day
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,61,28081.624,4,0.124031,0.054264,0.023256,0.038760,0.759690,2016,9,13
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,227,1388033.273,74,0.023680,0.004940,0.004429,0.003407,0.963543,2017,3,31
2,+++dz9ZCWE2HB/47pJU82NJXQzQuZDx1Wm50YSk/kKk=,54,27344.668,2,0.027523,0.000000,0.000000,0.009174,0.963303,2016,3,2
3,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,59,253460.349,39,0.165483,0.080256,0.065341,0.137784,0.551136,2017,3,31
4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,85,305064.873,64,0.060845,0.024338,0.028633,0.029349,0.856836,2017,3,31
...,...,...,...,...,...,...,...,...,...,...,...,...
2006679,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,7,1532.633,4,0.666667,0.125000,0.041667,0.083333,0.083333,2016,5,26
2006680,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,44,59525.679,11,0.040816,0.004082,0.004082,0.004082,0.946939,2017,3,28
2006681,zzztPAN9xjMytpZ0RN2gU9mScDULJnHQZK8eZb4uELU=,5,1251.328,4,0.473684,0.368421,0.052632,0.105263,0.000000,2015,2,9
2006682,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,5,5165.930,4,0.090909,0.000000,0.000000,0.181818,0.727273,2015,4,22


In [48]:
# percentage of number of songs played
total = (user_logs_gb['num_25']+user_logs_gb['num_50']+user_logs_gb['num_75']+user_logs_gb['num_985']+user_logs_gb['num_100'])
user_logs_gb['percent_25'] = user_logs_gb['num_25']/total
user_logs_gb['percent_50'] = user_logs_gb['num_50']/total
user_logs_gb['percent_75'] = user_logs_gb['num_75']/total
user_logs_gb['percent_985'] = user_logs_gb['num_985']/total
user_logs_gb['percent_100'] = user_logs_gb['num_100']/total

In [50]:
user_logs_gb = user_logs_gb.drop(columns = ['num_25', 'num_50', 'num_75', 'num_985', 'num_100'])

In [52]:
user_logs_gb = split_date(df=user_logs_gb,col_name='date')

In [54]:
user_logs_gb = user_logs_gb.drop(columns = ['date'])

In [57]:
#write_data(df=user_logs_gb, full_url=project_url+'Data/user_logs_gb.csv')

Saved csv in drive/MyDrive/DataScience/project/Data/user_logs_gb.csv


In [9]:
user_logs_gb = read_data(full_url=project_url+'Data/user_logs_gb.csv')

Upload csv from drive/MyDrive/DataScience/project/Data/user_logs_gb.csv


## Merge Data

In [10]:
# Merge train & user logs
train_merge = pd.merge(full_train, user_logs_gb, on='msno', how='left')

In [11]:
# Merge train & transcations
train_merge = pd.merge(train_merge, transactions_v3, on='msno', how='left')

In [12]:
# Merge train & members
train_merge = pd.merge(train_merge, members2, on='msno', how='left')

In [15]:
#write_data(df=train_merge, full_url=project_url+'Data/train_flat_file.csv')

Saved csv in drive/MyDrive/DataScience/project/Data/train_flat_file.csv


In [4]:
train_flat_file = read_data(full_url=project_url+'Data/train_flat_file.csv')

FileNotFoundError: [Errno 2] File drive/MyDrive/DataScience/project/Data/train_flat_file.csv does not exist: 'drive/MyDrive/DataScience/project/Data/train_flat_file.csv'

In [5]:
import socket;socket.gethostname()

'22c8ae54f996'

In [4]:
train_flat_file

NameError: ignored

In [22]:
# Merge test & transcations
test_merge = pd.merge(test, transactions_v3, on='msno', how='left')

In [24]:
# Merge test & members
test_merge = pd.merge(test_merge, members2, on='msno', how='left')

In [ ]:
# Merge test & user logs
train_merge = pd.merge(test_merge, user_logs_gb, on='msno', how='left')

In [ ]:
dataset_train['registration_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['registration_init_time']).astype('timedelta64[D]')
dataset_train['transaction_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['transaction_date']).astype('timedelta64[D]')
dataset_train['membership_expire_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['membership_expire_date']).astype('timedelta64[D]')
dataset_train['last_play_day'] = (dataset_train['membership_expire_date'].max() - dataset_train['date']).astype('timedelta64[D]')

# EDA

In [ ]:
figure(num=None, figsize=(16,8), dpi=60)
sns.countplot(x='payment_method_id', data=transaction_df)
xticks(rotation=90)

In [ ]:
sns.boxplot(x='plan_list_price', data=transaction_df)

In [ ]:
sns.countplot(x='is_auto_renew', data=transaction_df)